# 事前準備

### Googleドライブをマウントする

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

### import

In [0]:
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Activation, Flatten
import numpy as np
from google.colab import files
import pandas as pd
import io
import random
from keras.preprocessing.image import load_img, img_to_array
import os, sys, glob, cv2
from tqdm import tqdm as tqdm
from sklearn.model_selection import train_test_split
import zipfile
import shutil

# データの読み込み

## データの移動
Google DriveからGPUの場所へダウンロード

In [0]:
with zipfile.ZipFile('./gdrive/My Drive/data/curel/static.zip') as existing_zip:
  existing_zip.extractall('./')
if not os.path.exists("./picked_data"):
  os.mkdir("./picked_data")
shutil.copyfile("./gdrive/My Drive/data/curel/picked_data/positive.txt", "./picked_data/positive.txt")
shutil.copyfile("./gdrive/My Drive/data/curel/picked_data/negative.txt", "./picked_data/negative.txt")

In [0]:
in_origin = "."

## 画像の水増し
https://qiita.com/bohemian916/items/9630661cd5292240f8c7


In [0]:
def scratch_image(img, label, image_size):
  img = cv2.resize(img, (image_size,image_size))
  img = np.array(img)
  label = np.array(label)
  images = [img]
  labels = [label]

  # ルックアップテーブルの生成
#   min_table = 50
#   max_table = 205
#   diff_table = max_table - min_table

#   LUT_HC = np.arange(256, dtype='uint8')
#   LUT_LC = np.arange(256, dtype='uint8')
#   LUT_G1 = np.arange(256, dtype='uint8')
#   LUT_G2 = np.arange(256, dtype='uint8')

#   # ハイコントラストLUT作成
#   for i in range(0, min_table):
#       LUT_HC[i] = 0
#   for i in range(min_table, max_table):
#       LUT_HC[i] = 255 * (i - min_table) / diff_table
#   for i in range(max_table, 255):
#       LUT_HC[i] = 255
#   for i in range(256):
#       LUT_LC[i] = min_table + i * (diff_table) / 255
#   images.append(cv2.LUT(img, LUT_HC))
#   labels.append(label)
#   images.append(cv2.LUT(img, LUT_LC))
#   labels.append(label)
#   # image = np.concatenate((cv2.LUT(img, LUT_HC), img, cv2.LUT(img, LUT_LC)), axis = 1)

#   # ガンマ変換ルックアップテーブル
#   gamma1 = 0.75
#   gamma2 = 1.5
#   for i in range(256):
#       LUT_G1[i] = 255 * pow(float(i) / 255, 1.0 / gamma1)
#       LUT_G2[i] = 255 * pow(float(i) / 255, 1.0 / gamma2)
#   images.append(cv2.LUT(img, LUT_G1))
#   labels.append(label)
#   images.append(cv2.LUT(img, LUT_G2))
#   labels.append(label)
#   # image = np.concatenate((cv2.LUT(img, LUT_G1), img, cv2.LUT(img, LUT_G2)), axis = 1)

#   # 画像の平滑化
#   average_square = (10, 10)
#   images.append(cv2.blur(img, average_square))
#   labels.append(label)
#   # image = np.concatenate((cv2.blur(img, average_square), img), axis = 1)

#   # ガウス分布に基づくノイズ
#   # row, col, ch = img.shape
#   # mean = 0
#   # sigma = 15
#   # gauss = np.random.normal(mean, sigma, (row, col, ch))
#   # gauss = gauss.reshape(row, col, ch)
#   # gauss_img = img + gauss
#   # images.append(gauss_img)
#   # image = np.concatenate((gauss_img, img), axis = 1)

#   # salt & pepper ノイズ
#   row, col, ch = img.shape
#   s_vs_p = 0.5
#   amount = 0.004
#   sp_img = img.copy()

#   # 塩モード
#   num_salt = np.ceil(amount * img.size * s_vs_p)
#   coords = [np.random.randint(0, i-1, int(num_salt)) for i in img.shape]
#   sp_img[coords[:-1]] = (255, 255, 255)

#   # 故障モード
#   num_pepper = np.ceil(amount * img.size * (1. - s_vs_p))
#   coords = [np.random.randint(0, i-1, int(num_pepper)) for i in img.shape]
#   sp_img[coords[:-1]] = (0, 0, 0)
#   images.append(sp_img)
#   labels.append(label)
#   # image = sp_img

#   # 反転
#   hflip_img = cv2.flip(img, 1)
#   vflip_img = cv2.flip(img, 0)
#   images.append(hflip_img)
#   labels.append([label[0], label[1], label[2], label[3], image_size-label[4], label[5], label[6], label[7]])
#   images.append(vflip_img)
#   labels.append([label[0], label[1], label[2], label[3], label[4], image_size-label[5], label[6], label[7]])
  # image = np.concatenate((hflip_img, vflip_img), axis = 1)

  # 拡大縮小
#   hight = int(img.shape[0] / 2)
#   width = int(img.shape[1] / 2)
#   half_img = cv2.resize(img, (hight, width))
#   labels.append([label[0], label[1], label[2], label[3], label[4],label[4] / 2, label[5] / 2, label[6] / 2, label[7] / 2])
#   images.append(half_img)
  # image = half_img

  # plt.imshow(image)
  # plt.show()

  return (images, labels)

## ダウンロード

[どの可能性も低い, カテゴリー1, カテゴリー2, カテゴリー3, 座標x1, 座標y1, 座標x2, 座標y2]
x2, y2はx1,y1からの距離

In [71]:
image_size = 128
positive = open(in_origin + "/picked_data/positive.txt", "r")
negative = open(in_origin + "/picked_data/negative.txt", "r")

data_x = []
data_y = []

for line in tqdm(positive.readlines(), "positive"):
#   前処理
  line = line.split()
  line[1] = line[1].replace('\\', '/')
  
#   画像読み込み
  image = cv2.imread(str(in_origin + '/' + line[1]))
  if image is None:
    print("none" + str(in_origin + '/' + line[1]))
    continue
  
#   ラベリング
  for i in range(int(line[2])):
    label = [0, 1, float(line[i*2+3]), float(line[i*2+4])]
#     (new_images, new_labels) = scratch_image(image, label, image_size)
#     for new_image in new_images:
#       data_x.append(new_image)
#     for new_label in new_labels:
#       data_y.append(new_label)
    data_x.append(image)
    data_y.append(label)


for line in tqdm(negative.readlines(), "negative"):
  line = line.split()
  line[0] = line[0].replace('\\', '/')
  image = cv2.imread(str(in_origin + '/' + line[0]))
  if image is None:
    print("none" + str(in_origin + '/' + line[0]))
    continue
  
  image = cv2.resize(image, (image_size, image_size))
  label = [1, 0, 0, 0, 0, 0]
#   (new_images, new_labels) = scratch_image(image, label, image_size)
#   for new_image in new_images:
  data_x.append(image)
#   for new_label in new_labels:
  data_y.append(label)

negative: 100%|██████████| 37/37 [00:00<00:00, 2128.36it/s]


In [72]:
test_size = 0.1

x_train = []
y_train = []
x_test = []
y_test = []
for i in range(int(len(data_x))):
  if len(data_x[i]) == 64:
    continue
  if random.randrange(1000) < test_size * 1000:
#     テスト用
    label = [data_y[i][0], data_y[i][1], data_y[i][2] / 128, data_y[i][3] / 128]
    x_test.append(data_x[i])
    y_test.append(label)
  else:
    label = [data_y[i][0], data_y[i][1], data_y[i][2] / 128, data_y[i][3] / 128]
    x_train.append(data_x[i])
    y_train.append(label)
#   print(data_x[i].shape)
print("train : " + str(len(x_train)) + " test : " + str(len(x_test)))
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
print(x_train.shape)
x_train = x_train / 255
x_test = x_test / 255
print(x_train.shape)
del data_x, data_y

train : 98 test : 13
(98, 128, 128, 3)
(98, 128, 128, 3)


In [73]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
# print(x_train[0])
print(y_train[0])
# print(x_test[0])
print(y_test[0])

(98, 128, 128, 3)
(98, 4)
(13, 128, 128, 3)
(13, 4)
[0.         1.         0.32317708 0.46851852]
[0.         1.         0.32447917 0.46666667]


In [0]:
model = Sequential()

model.add(Conv2D(64,(3,3),padding='same',input_shape=(128,128,3)))
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=((2,2))))
model.add(Dropout(0.2))

model.add(Conv2D(128,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(256,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

# def mymetrics

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

In [78]:
history = model.fit(x_train, y_train, batch_size=256, epochs=100, verbose=1, validation_split=0.1)

Train on 88 samples, validate on 10 samples
Epoch 1/100
88/88 [==============================] - 1s 8ms/step - loss: 1.6441 - acc: 0.8750 - val_loss: 4.5292 - val_acc: 0.2000
Epoch 2/100
88/88 [==============================] - 1s 6ms/step - loss: 1.6346 - acc: 0.8750 - val_loss: 4.6012 - val_acc: 0.3000
Epoch 3/100
88/88 [==============================] - 1s 6ms/step - loss: 1.6357 - acc: 0.8523 - val_loss: 4.6835 - val_acc: 0.4000
Epoch 4/100
88/88 [==============================] - 1s 6ms/step - loss: 1.6308 - acc: 0.8750 - val_loss: 4.6777 - val_acc: 0.4000
Epoch 5/100
88/88 [==============================] - 1s 6ms/step - loss: 1.6178 - acc: 0.8750 - val_loss: 4.3132 - val_acc: 0.4000
Epoch 6/100
88/88 [==============================] - 1s 6ms/step - loss: 1.6136 - acc: 0.8409 - val_loss: 4.3438 - val_acc: 0.4000
Epoch 7/100
88/88 [==============================] - 1s 6ms/step - loss: 1.5911 - acc: 0.8409 - val_loss: 4.4928 - val_acc: 0.4000
Epoch 8/100
88/88 [====================

In [79]:
score = model.evaluate(x_test, y_test)
print(score)

13/13 [==============================] - 0s 4ms/step
[2.5998411178588867, 0.7692307829856873]


In [84]:
np.set_printoptions(suppress=True)
result = model.predict(np.array([x_test[4]]))
print(result)
print(y_test[4])

[[0.0664491  0.5136571  0.1902337  0.22966012]]
[0.         1.         0.45351563 0.52083333]


In [0]:
open('model.json', 'w').write(model.to_json())
model.save_weights('param.hdf5')
files.download('model.json')
files.download('param.hdf5')